In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
from time import sleep,time
from random import randint
from IPython.core.display import clear_output
import json

In [2]:
import string
import re
import gensim
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize

In [23]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
harry_corpus=[]

In [22]:
harry_books=["Harry-Potter-and-the-Philosophers-Stone","Harry-Potter-and-the-Chamber-of-Secrets","Harry-Potter-and-the-Goblet-of-Fire",
             "Harry-Potter-and-the-Prisoner-of-Azkaban","Harry-Potter-and-the-Order-of-the-Phoenix","Harry-Potter-and-the-Half-Blood-Prince",
            "Harry-Potter-and-the-Deathly-Hallows"]

In [44]:
book=harry_books[6]

In [45]:


start_time = time()
requests = 0

response=get("https://www.bookscool.com/en/"+str(book)+"/1")
html = BeautifulSoup(response.text,'html.parser')
page=html.find(class_="nav nav-underline")
last_page=int(page.findAll('a')[-1].text)

corpus=""
for i in range(0,last_page+1):
        response=get("https://www.bookscool.com/en/"+str(book)+"/"+str(i)+"")
        html = BeautifulSoup(response.text,'html.parser')    
        pages=html.find(class_="text-muted pt-3")
        pages_text=pages.findAll('br')[2:-1]
        
        requests+=1
        elapsed_time=time()-start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)
        
        vachan=""
        for i in range(len(pages_text)):
            vachan+=pages_text[i].text+" "
        corpus+=vachan+" "

Request:37; Frequency: 1.1039295879723432 requests/s


In [46]:
harry_corpus.append(corpus)  

In [48]:
corpus=" ".join(harry_corpus)

In [62]:
clean_text=re.sub(r"[,;@#?“”!&\"\")($\'’]+\ *", " ",corpus)
# clean_text=re.sub(r"[\"\"]+\ *"," ",clean_text)

In [64]:
data=[]
for i in sent_tokenize(clean_text):
    data1=[]
    for j in word_tokenize(i):
        if j not in stop_words:
            data1.append(j.lower())
    data.append(data1)

In [73]:
model = gensim.models.Word2Vec(size=100, window=5, min_count=2, sg=1, workers=10)
model.build_vocab(data)

In [74]:
model.train(sentences=data, total_examples=len(data), epochs=model.iter)

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(124220, 148840)

In [75]:
w1 = "ron"
model.wv.most_similar(positive=w1, topn=10)

[('gone', 0.9992308020591736),
 ('anyone', 0.9991603493690491),
 ('lot', 0.9991506934165955),
 ('let', 0.999133825302124),
 ('loudly', 0.9991320967674255),
 ('kept', 0.9991299510002136),
 ('letters', 0.9991264343261719),
 ('send', 0.9991152286529541),
 ('followers', 0.9991083145141602),
 ('grown', 0.999108076095581)]